### Opis danych we wszystkich plikach
- 'age', - wiek klienta
- 'account_name', - imie klienta
- 'account_surname', - nazwisko klienta
- 'product', - produkt
- 'order_city', - miasto z ktorego zlozono zamowienie
- 'color', - kolor produktu
- 'date', - data produkcji
- 'delivery_date', - data dostarczenia
- 'production_date', - data zamowienia
- 'discount', - upust %
- 'ordered', - ilosc zamowiona
- 'quantity', - ilosc dostarczona 
- 'Invoice_Id', - numer faktury, klucz
- 'region', - region
- 'additional_delivery_cost', - oznaczenie typu dostawy i kosztu dostawy
- 'wrh_city', - relacja magazyn z ktorego wyslano towar i destynacja zamowienia
- 'price' - cena
- 'City' - miasto
- 'Zone' - region

---
'additional_delivery_cost'
- T - truck - koszt na sztuke dostarczona (0.15 per sztuka)
- C - container - koszt na sztuke zadeklarowana/zamowiona (0.11 per sztuka)
- F - flight - koszt na sztuke dostarczona (0.82 per sztuka)
- ? - brak danych


**1. Zaimportuj potrzebne biblioteki**

In [1]:
import numpy as np
import pandas as pd
import pyarrow


**2. Zaimportuj dane**

Zaimportuj dane z plikow:
- ZadaniaDf1 - csv
- ZadaniaDf2.parquet - parquet
- ZadaniaDf3 - txt
- ZadaniaDf4 - xlsx

Nazwy DataFrameow powinny miec odpowiednio nazwy df1,df2,df3,df4
Dane powinny być zaimportowane w taki sposób aby kolumny posiadaly odpowiednie nazwy oraz unikalny indeks na poczatku

In [4]:
df1 = pd.read_csv(r"C:\PythonWorkshop\wsbpythonfiles\Zadania\ZadaniaDf1.csv",sep='/')

In [6]:
df2 = pd.read_parquet(path=r"C:\PythonWorkshop\wsbpythonfiles\Zadania\ZadaniaDf2.parquet")

In [10]:
df3 = pd.read_csv(r"C:\PythonWorkshop\wsbpythonfiles\Zadania\ZadaniaDf3.txt", sep=' ')

In [13]:
df4 = pd.read_excel(r"C:\PythonWorkshop\wsbpythonfiles\Zadania\ZadaniaDf4.xlsx",sheet_name="Arkusz1")

In [14]:
df4.head(15)

,City,Zone
0,Stockholm,Europe
1,Denver,America
2,Moscow,Asia
3,Marseille,Europe
4,Palermo,Europe
5,Tokyo,Asia
6,Lisbon,Europe
7,Oslo,Europe
8,Nairobi,Africa
9,Río de Janeiro,America


**3. Dokonaj rewizji danych**

Dokonaj wstępnej rewizji każdego DataFrameu:
- Przejrzyj kilka pierwszych i ostatnich rekordow kazdego zestawu danych
- Zweryfikuj czy w danych pojawiaja sie nulle oraz ile ich jest (Zwroc uwage na dane z formatu parquet)
- Zidentyfikuj ewentualne klucze w celu polaczenia zestawow
- Zidentyfikuj czy dane sie duplikuja (kolumny)

**4. Nadpisz datasety**
- Nadpisz wszystkie datasety tak aby posiadaly wylacznie unikalne kolumny (brak duplikatow - nalezy wybrac kolumny albo zastosowac drop). 
- Pamietaj aby nie usunac kolumn ktore sa zidentyfikowane jako klucze. 
- Nadaj prefix "u_" dla kazdego datasetu. Nawet jesli dataset nie jest redukowany powinien otrzymac prefix.

**5. Polacz datasety**
- Polacz ze soba datasety. 
- Dolaczaj do siebie datasety df1 do df2 etc. Kazde kolejne polaczenie nadpisuje pod nazwa "u_df1_merge". 
- Po skonczonych polaczeniach usun niepotrzebna kolumne "City" i zapisz dataset jako "merged_df"

**6. Zapisz dane do pliku, wykonaj ponowna analize danych i pogleb ja**
- Zapisz dane do pliku .csv pod nazwa "merged_df". 
- Zrestartuj kernel aby zwolnic pamiec ram. 
- Odczytaj zapisany wczesniej plik i dokonaj jego analizy, sprawdz czy wczesniej ustalone kolumny z pustymi wartosciami sa takie same czy cos sie zmienilo
- Wstepna wizualizacja danych przy tak duzej ilosci danych moze byc nieskuteczna/nieczytelna. Dlatego poleca sie zweryfikowac unikalne wartosci dla kazdej z kolumn. W przypadku ograniczenia widocznosci poleca sie przekonwertowac dane na liste.
- Mozna pominac klucz po ktorym laczono dane poniewaz jest unikalny 

**7. Wyczysc dane I - usuwanie**
Po konsultacji z odbiorcami danych ustalono:
- "product" - nalezy usunac wszystkie rekordy ktore w kolumnie posiadaja puste wartosci, bez tego analiza sprzedazy bedzie skrzywiona
- "quantity" - nalezy usunac wszystkie rekordy ktore w kolumnie posiadaja ujemne wartosci, bez tego analiza sprzedazy bedzie skrzywiona, dane obarczone bledami
- "region" - nalezy usunac kolumne gdyz jest pusta

**8. Wyczysc dane II - zastepowanie pustych**
Po konsultacji z odbiorcami danych ustalono ze puste w:
- "account_name" - uzupelniamy "John"
- "account_surname" - uzupelniamy "Doe"
- "order_city" - uzupelniamy najczestszym wystapieniem w kolumnie dla niepustych
- "discount" - uzupelniamy "0.0"
- "Zone" - kasujemy kolumne "zone",mergujemy ponownie data set z df4, nowa kolumna Zone powinna byc juz uzupelniona po wypelnieniu "order_city", kasujemy kolune "City", dataset nadpisujemy z ta sama nazwa

**9. Wyczysc dane III - podmiana danych, zmiana typow**
Po konsultacji z odbiorcami danych ustalono ze zamieniamy dane:
- "age" - dane powyzej 100 lat sa poprawne ale pomnozone przez 10, nalezy je podzielic przez 10
- "account_name" - nalezy zkapitalizowac
- "account_surname" - nalezy usunac prefixy "Old_"
- "color" - nalezy zkapitalizowac
- "production_date" - kolumne nalezy przekonwertowac na datetime "RRRR-MM-DD"
- "date" - kolumne nalezy przekonwertowac na datetime "RRRR-MM-DD"
- "delivery_date" - kolumne nalezy przekonwertowac na datetime "RRRR-MM-DD"
- "wrh_city" - zostawiamy tylko pierwszy czlon przed "_" - wrh
- "discount" - kolumne nalezy przekonwertowac na float

Wszystkie zmiany wykonujemy funkcjami z pozimu serii/kolumny za wyjatkiem konwertowania kolumn

**10. Dodawanie kolumn**
Po konsultacji z odbiorcami danych ustalono zamieniamy dane:
- "cost" - tworzymy kolumne wartoscia kosztow na podstawie opisu kolumny 'additional_delivery_cost' np. jezeli w kolumnie mamy TC to mnozymy wartosc quantity x 0.15(T) i dodajemy ordered x 0.11(C), w przypadku "?" stosujemy wartosc ((0.15+0.11+0.82)/3) x quantity. Dodatkowo kazdego wyniku dodajemy koszty pozostale na poziomie 1.5(nie mnozymy przez ilosci, jednorazowa kwota)
- "revenue" - tworzymy kolumne ktora jest wynikiem quantity x (price-(price x discount))
- "margin" - revenue - cost
- "order_time" - delivery_time - production_date
- "lost_revenue" - tworzymy kolumne ktora jest wynikiem (ordered - quantity) x (price-(price x discount))
- "year" - kolumna z rokiem na podstawie konwersji production_date

Wszystkie zmiany wykonujemy funkcjami z poziomu DataFrameu za wyjatkiem konwersji

**11. Redukcja danych zbednych danych i wizualna korekcja**
Po konsultacji z odbiorcami danych ustalono:
- kasujemy:
    - account_name
    - account_surname
    - Invoice_Id
    - date
    - delivery_date
    - discount
    - ordered
    - quantity
    - additional_delivery_cost
    - price
- zmieniamy nazwy kolumn:
    - "age":"Wiek",
    - "product":"Produkt",
    - "order_city":"Miasto_zamowienia",
    - "color":"Kolor",
    - "production_date":"Data_zamowienia",
    - "wrh_city":"Magazyn",
    - "Zone":"Region",
    - "revenue":"Przychow",
    - "cost":"Koszt"
    - "margin":"Marza",
    - "order_time":"Czas_dostawy",
    - "lost_revenue":"Utracony_przychow"

Wszystkie zmiany wykonujemy funkcjami z poziomu DataFrameu

**12. Wizualizacje pogladowe**
W trakcie przetwarzania danych dzial sprzedazy poprosil Ciebie o pokazanie 3 szybkich widokow danych
 - wykresu slupkowego (pionowego) na ktorym znajduja sie sumy Przychodu i Utraconego przychodu dla wszystkich lat, wykres powinien posiadac siatke (aby wykonac wykres nalezy zgrupowac Przychod i Utracony przychod po Roku, zsumowac a nastepnie wygenerowac wykres)
 - wykresu slupkowego (pionowego) na ktorym znajduje sie sumy Przychodu dla top 5 produktow (aby wykonac wykres nalezy zgrupowac Przychod po Produkcie, zsumowac, wylistowac top 5, a nastepnie wygenerowac wykres)
 - wykresu slupkowego (poziomego) na ktorym znajduja sie sumych Przychodu dla Wieku i Produktu dla top 5 zgrupowan tych elementow (aby wykonac wykres nalezy zgrupowac Przychod po Wieku i Produkcie, zsumowac, wylistowac top 5 a nastepnie wygenerowac wykres)

Wszystkie wykresy sa pogladowe, wiec nie ma koniecznosci zmiany ich oprawy wizualnej